In [ ]:
import os
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import rioxarray
import rasterio

# Define output directory
output_dir = 'D:/Data Science/Geographycal Analysis/Output'
os.makedirs(output_dir, exist_ok=True)

# Load NetCDF data
aet_data = xr.open_dataset(r'D:\Data Science\Geographycal Analysis\Data/AET_data.nc')
precipitation_data = xr.open_dataset(r'D:\Data Science\Geographycal Analysis\Data/Precipt_data.nc')
temperature_data = xr.open_dataset(r'D:\Data Science\Geographycal Analysis\Data/Tp_data.nc')
windspeed_data = xr.open_dataset(r'D:\Data Science\Geographycal Analysis\Data/Ws_data.nc')

# Assuming the CRS is EPSG:4326 (WGS84). Adjust if necessary.
crs = 'EPSG:4326'

# Set the CRS for each dataset
aet_data.rio.write_crs(crs, inplace=True)
precipitation_data.rio.write_crs(crs, inplace=True)
temperature_data.rio.write_crs(crs, inplace=True)
windspeed_data.rio.write_crs(crs, inplace=True)

# Load Kenya's geometry
kenya_geometry = gpd.read_file('D:/Data Science/Geographycal Analysis/Shp_files/ESRI_countries.shp')

# Convert Kenya's geometry to a mask
kenya_mask = kenya_geometry.geometry.apply(mapping)

def clip_xarray_to_geometry(ds, geometry):
    return ds.rio.clip(geometry, all_touched=True, drop=True)

aet_clipped = clip_xarray_to_geometry(aet_data, kenya_mask)
precipitation_clipped = clip_xarray_to_geometry(precipitation_data, kenya_mask)
temperature_clipped = clip_xarray_to_geometry(temperature_data, kenya_mask)
windspeed_clipped = clip_xarray_to_geometry(windspeed_data, kenya_mask)

# Save clipped data for further analysis
aet_clipped.to_netcdf(os.path.join(output_dir, 'aet_clipped.nc'))
precipitation_clipped.to_netcdf(os.path.join(output_dir, 'precipitation_clipped.nc'))
temperature_clipped.to_netcdf(os.path.join(output_dir, 'temperature_clipped.nc'))
windspeed_clipped.to_netcdf(os.path.join(output_dir, 'windspeed_clipped.nc'))

# Load clipped data
aet_clipped = xr.open_dataset(os.path.join(output_dir, 'aet_clipped.nc'))
precipitation_clipped = xr.open_dataset(os.path.join(output_dir, 'precipitation_clipped.nc'))
temperature_clipped = xr.open_dataset(os.path.join(output_dir, 'temperature_clipped.nc'))
windspeed_clipped = xr.open_dataset(os.path.join(output_dir, 'windspeed_clipped.nc'))

# Convert data to DataFrame for easier manipulation
aet_df = aet_clipped.to_dataframe().reset_index()
precipitation_df = precipitation_clipped.to_dataframe().reset_index()
temperature_df = temperature_clipped.to_dataframe().reset_index()
windspeed_df = windspeed_clipped.to_dataframe().reset_index()

# Descriptive statistics
aet_stats = aet_df.describe()
precipitation_stats = precipitation_df.describe()
temperature_stats = temperature_df.describe()
windspeed_stats = windspeed_df.describe()

# Displaying statistics
print("AET Statistics:\n", aet_stats)
print("Precipitation Statistics:\n", precipitation_stats)
print("Temperature Statistics:\n", temperature_stats)
print("Windspeed Statistics:\n", windspeed_stats)

# Histograms
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

axs[0, 0].hist(aet_df['evavt'], bins=30, color='blue', alpha=0.7)
axs[0, 0].set_title('AET Distribution')

axs[0, 1].hist(precipitation_df['tp'], bins=30, color='green', alpha=0.7)
axs[0, 1].set_title('Precipitation Distribution')

axs[1, 0].hist(temperature_df['t2m'], bins=30, color='red', alpha=0.7)
axs[1, 0].set_title('Temperature Distribution')

axs[1, 1].hist(windspeed_df['u10'], bins=30, color='orange', alpha=0.7)
axs[1, 1].set_title('Windspeed Distribution')

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'histograms.png'))
plt.show()

# Convert 'time' column to datetime
aet_df['time'] = pd.to_datetime(aet_df['time'])
precipitation_df['time'] = pd.to_datetime(precipitation_df['time'])
temperature_df['time'] = pd.to_datetime(temperature_df['time'])
windspeed_df['time'] = pd.to_datetime(windspeed_df['time'])

# Calculate monthly averages
aet_monthly_avg = aet_df.groupby(aet_df['time'].dt.to_period('M')).mean()
precipitation_monthly_avg = precipitation_df.groupby(precipitation_df['time'].dt.to_period('M')).mean()
temperature_monthly_avg = temperature_df.groupby(temperature_df['time'].dt.to_period('M')).mean()
windspeed_monthly_avg = windspeed_df.groupby(windspeed_df['time'].dt.to_period('M')).mean()

# Plotting the monthly series
fig, axs = plt.subplots(4, 1, figsize=(12, 18))

aet_monthly_avg['evavt'].plot(ax=axs[0], title='Monthly Average AET')
axs[0].set_ylabel('AET')

precipitation_monthly_avg['tp'].plot(ax=axs[1], title='Monthly Average Precipitation')
axs[1].set_ylabel('Precipitation')

temperature_monthly_avg['t2m'].plot(ax=axs[2], title='Monthly Average Temperature')
axs[2].set_ylabel('Temperature')

windspeed_monthly_avg['u10'].plot(ax=axs[3], title='Monthly Average Windspeed')
axs[3].set_ylabel('Windspeed')

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'monthly_averages.png'))
plt.show()

# Scatter plots
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

axs[0].scatter(aet_monthly_avg['evavt'], precipitation_monthly_avg['tp'])
axs[0].set_xlabel('AET')
axs[0].set_ylabel('Precipitation')
axs[0].set_title('AET vs Precipitation')

axs[1].scatter(aet_monthly_avg['evavt'], temperature_monthly_avg['t2m'])
axs[1].set_xlabel('AET')
axs[1].set_ylabel('Temperature')
axs[1].set_title('AET vs Temperature')

axs[2].scatter(aet_monthly_avg['evavt'], windspeed_monthly_avg['u10'])
axs[2].set_xlabel('AET')
axs[2].set_ylabel('Windspeed')
axs[2].set_title('AET vs Windspeed')

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'scatter_plots.png'))
plt.show()

# Correlation coefficients
precip_temp_corr = np.corrcoef(precipitation_monthly_avg['tp'], temperature_monthly_avg['t2m'])[0, 1]
precip_wind_corr = np.corrcoef(precipitation_monthly_avg['tp'], windspeed_monthly_avg['u10'])[0, 1]

print(f"Correlation between Precipitation and Temperature: {precip_temp_corr}")
print(f"Correlation between Precipitation and Windspeed: {precip_wind_corr}")

# Function to create a custom boxplot for each year
def create_custom_boxplot_per_year(data, variable, title, ylabel):
    fig, ax = plt.subplots(figsize=(14, 5))
    years = data['time'].dt.year.unique()
    data_list = [data[data['time'].dt.year == year][variable].dropna() for year in years]
    
    my_flierprops = dict(markerfacecolor='g', markeredgecolor='g', marker='+')
    my_medianprops = dict(color='r', linewidth=2)
    my_boxprops = dict(facecolor='b', edgecolor='b')
    
    ax.boxplot(data_list, patch_artist=True, notch=True,
               flierprops=my_flierprops, medianprops=my_medianprops, boxprops=my_boxprops)
    
    ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.set_xticks(range(1, len(years) + 1))
    ax.set_xticklabels(years, rotation=45)
    plt.savefig(os.path.join(output_dir, f'{title}.png'))
    plt.show()

# Create custom boxplots for each year
create_custom_boxplot_per_year(aet_df, 'evavt', 'Annual Average AET', 'AET [units]')
create_custom_boxplot_per_year(precipitation_df, 'tp', 'Annual Average Precipitation', 'Precipitation [units]')
create_custom_boxplot_per_year(temperature_df, 't2m', 'Annual Average Temperature', 'Temperature [units]')
create_custom_boxplot_per_year(windspeed_df, 'u10', 'Annual Average Windspeed', 'Windspeed [units]')

# Define output directory
output_dir = 'D:/Data Science/Geographycal Analysis/Output'

# Load the clipped NetCDF files
aet_clipped = xr.open_dataset(os.path.join(output_dir, 'aet_clipped.nc'))
precipitation_clipped = xr.open_dataset(os.path.join(output_dir, 'precipitation_clipped.nc'))
temperature_clipped = xr.open_dataset(os.path.join(output_dir, 'temperature_clipped.nc'))
windspeed_clipped = xr.open_dataset(os.path.join(output_dir, 'windspeed_clipped.nc'))

# Define the CRS (assuming EPSG:4326 for this example)
crs = 'EPSG:4326'

# Set the CRS for each dataset
aet_clipped.rio.write_crs(crs, inplace=True)
precipitation_clipped.rio.write_crs(crs, inplace=True)
temperature_clipped.rio.write_crs(crs, inplace=True)
windspeed_clipped.rio.write_crs(crs, inplace=True)

# Load Kenya's geometry
kenya_geometry = gpd.read_file('D:/Data Science/Geographycal Analysis/Shp_files/ESRI_countries.shp')
kenya_geometry = kenya_geometry[kenya_geometry['CNTRY_NAME'] == 'Kenya']

# Define the time steps (ensure these are within the dataset's time range)
time_step_1 = '2007-01-01'
time_step_2 = '2020-01-01'

# Calculate monthly maximum
aet_max = aet_clipped.resample(time='M').max()
precipitation_max = precipitation_clipped.resample(time='M').max()
temperature_max = temperature_clipped.resample(time='M').max()
windspeed_max = windspeed_clipped.resample(time='M').max()

# Select the specific time steps using the nearest method
# This ensures we get the closest available data point if the exact time step isn't present
aet_max_1 = aet_max.sel(time=time_step_1, method='nearest')
aet_max_2 = aet_max.sel(time=time_step_2, method='nearest')
precipitation_max_1 = precipitation_max.sel(time=time_step_1, method='nearest')
precipitation_max_2 = precipitation_max.sel(time=time_step_2, method='nearest')
temperature_max_1 = temperature_max.sel(time=time_step_1, method='nearest')
temperature_max_2 = temperature_max.sel(time=time_step_2, method='nearest')
windspeed_max_1 = windspeed_max.sel(time=time_step_1, method='nearest')
windspeed_max_2 = windspeed_max.sel(time=time_step_2, method='nearest')

# Clip the data to Kenya's geometry
aet_max_1 = aet_max_1.rio.clip(kenya_geometry.geometry, kenya_geometry.crs)
aet_max_2 = aet_max_2.rio.clip(kenya_geometry.geometry, kenya_geometry.crs)
precipitation_max_1 = precipitation_max_1.rio.clip(kenya_geometry.geometry, kenya_geometry.crs)
precipitation_max_2 = precipitation_max_2.rio.clip(kenya_geometry.geometry, kenya_geometry.crs)
temperature_max_1 = temperature_max_1.rio.clip(kenya_geometry.geometry, kenya_geometry.crs)
temperature_max_2 = temperature_max_2.rio.clip(kenya_geometry.geometry, kenya_geometry.crs)
windspeed_max_1 = windspeed_max_1.rio.clip(kenya_geometry.geometry, kenya_geometry.crs)
windspeed_max_2 = windspeed_max_2.rio.clip(kenya_geometry.geometry, kenya_geometry.crs)

# Create spatial plots
variables = {
    'AET': [aet_max_1['evavt'], aet_max_2['evavt']],
    'Precipitation': [precipitation_max_1['tp'], precipitation_max_2['tp']],
    'Temperature': [temperature_max_1['t2m'], temperature_max_2['t2m']],
    'Windspeed': [windspeed_max_1['u10'], windspeed_max_2['u10']]
}

time_steps = [time_step_1, time_step_2]

for var_name, data in variables.items():
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    for i, ax in enumerate(axes):
        data_array = data[i]
        im = data_array.plot(ax=ax, cmap='viridis')
        ax.set_title(f'{var_name} - {data[i]["time"].values}')
    plt.suptitle(f'{var_name} Monthly Maximum')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{var_name}_monthly_maximum.png'))
    plt.show()